In [ ]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
credit_card_data = spark.read.csv(path='credit card.csv', header=True)

In [ ]:
loan_data = spark.read.csv(path='loan.csv', header=True)

In [ ]:
txn_data = spark.read.csv(path='txn.csv', header=True)

In [ ]:
credit_card_data.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|        0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|        0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

In [ ]:
loan_data.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

In [ ]:
txn_data.show()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|      1000000|    1000000|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|      1000000|    2000000|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|       500000|    2500000|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|      3000000|    5500000|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|       500000|    6000000|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|       500000|    6500000|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|       500000|    7000000|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|       500000|    7500000|

1. Basic Setup

In [ ]:
# Registering DataFrames as temporary views
credit_card_data.createOrReplaceTempView("credit_card")
loan_data.createOrReplaceTempView("loan")
txn_data.createOrReplaceTempView("transaction")

2. Joins

a) Inner Join
Join credit_card_data with loan_data using CustomerId and Customer_ID as the key.

In [ ]:
inner_join = credit_card_data.join(loan_data, credit_card_data.CustomerId == loan_data.Customer_ID, "inner")
inner_join.show()

+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
+---------+----------+-------+-----------+---------+------+---+------+------

In [ ]:
query = """
SELECT *
FROM credit_card c
INNER JOIN loan l
ON c.CustomerId = l.Customer_ID
"""

spark.sql(query).show()

+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
+---------+----------+-------+-----------+---------+------+---+------+------

b) Left Join
Perform a left join between credit_card_data and loan_data.

In [ ]:
left_join = credit_card_data.join(loan_data, credit_card_data.CustomerId == loan_data.Customer_ID, "left")
left_join.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID| Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|        1|  15634602| Hargrave|        619|   France|Female|

In [ ]:
query = """
SELECT *
FROM credit_card c
LEFT JOIN loan l
ON c.CustomerId = l.Customer_ID
"""

spark.sql(query).show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID| Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|        1|  15634602| Hargrave|        619|   France|Female|

c) Right Join
Perform a right join between credit_card_data and loan_data.

In [ ]:
right_join = credit_card_data.join(loan_data, credit_card_data.CustomerId == loan_data.Customer_ID, "right")
right_join.show()

+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender| Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|     NULL|      NULL|   NULL| 

In [ ]:
query = """
SELECT *
FROM credit_card c
RIGHT JOIN loan l
ON c.CustomerId = l.Customer_ID
"""

spark.sql(query).show()

+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender| Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|     NULL|      NULL|   NULL| 

d) Full Outer Join
Perform a full outer join between credit_card_data and loan_data.

In [ ]:
full_outer_join = credit_card_data.join(loan_data, credit_card_data.CustomerId == loan_data.Customer_ID, "outer")
full_outer_join.show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID| Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|     1288|  15565701|     Ferri|        698|    Spain|Fem

In [ ]:
query = """
SELECT *
FROM credit_card c
FULL OUTER JOIN loan l
ON c.CustomerId = l.Customer_ID
"""

spark.sql(query).show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID| Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-----------+----+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|     1288|  15565701|     Ferri|        698|    Spain|Fem

e) Cross Join
Perform a cross join between credit_card_data and loan_data.

In [ ]:
cross_join = credit_card_data.crossJoin(loan_data)
cross_join.show()

+---------+----------+--------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+--------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|        1|  15634602|Hargrave|

In [ ]:
query = """
SELECT *
FROM credit_card c
CROSS JOIN loan l
"""

spark.sql(query).show()

+---------+----------+--------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+---------+----------+--------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|        1|  15634602|Hargrave|

3. Filters


Filter loan_data for customers with a Expenditure greater than 15,000

In [ ]:
filtered_loan = loan_data.filter(
    (loan_data['Expenditure'] > 15000)
)
filtered_loan.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

In [ ]:
query = """
SELECT *
FROM loan
WHERE Expenditure > 15000
"""

spark.sql(query).show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

Filter Customers with Balance Greater than 5000 and Active Member

In [ ]:
# Filter customers with Balance > 5000 and IsActiveMember = 1 (active member)
filtered_data = credit_card_data.filter(
    (credit_card_data['Balance'] > 5000) & (credit_card_data['IsActiveMember'] == 1)
)
filtered_data.show()

+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|      Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|         Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888|     Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        9|  15792365|           He|        501|   France|  Male| 44|     4|142051.07|            2|             1|        74940.5|     0|
|       10|  15592389|           H?|        684|   France|  Male| 27|     2|134603.88|            1|             1|       71725.73|     0|
|       16|  15643966|     

In [ ]:
query = """
SELECT *
FROM credit_card
WHERE Balance > 5000 AND IsActiveMember = 1
"""
spark.sql(query).show()

+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|      Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|         Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888|     Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        9|  15792365|           He|        501|   France|  Male| 44|     4|142051.07|            2|             1|        74940.5|     0|
|       10|  15592389|           H?|        684|   France|  Male| 27|     2|134603.88|            1|             1|       71725.73|     0|
|       16|  15643966|     

4. GroupBy and Aggregations
Find the total loan amount and average income by loan category.

In [ ]:
grouped_loan = loan_data.groupBy(['Loan Category']).agg(
    {'Loan Amount': "sum", "Income": "avg"}
)
grouped_loan.show()

+------------------+------------------+----------------+
|     Loan Category|       avg(Income)|sum(Loan Amount)|
+------------------+------------------+----------------+
|           HOUSING| 74728.19354838709|            NULL|
|        TRAVELLING| 57016.58490566038|            NULL|
|       BOOK STORES|50903.142857142855|            NULL|
|       AGRICULTURE|60372.666666666664|            NULL|
|         GOLD LOAN| 70838.31506849315|            NULL|
|  EDUCATIONAL LOAN| 62057.64705882353|            NULL|
|        AUTOMOBILE| 68285.63157894737|            NULL|
|          BUSINESS| 70246.54166666667|            NULL|
|COMPUTER SOFTWARES|134376.66666666666|            NULL|
|           DINNING| 67617.54545454546|            NULL|
|          SHOPPING|       50466.34375|            NULL|
|       RESTAURANTS|55228.794871794875|            NULL|
|       ELECTRONICS| 54728.42857142857|            NULL|
|          BUILDING| 69700.16666666667|            NULL|
|        RESTAURANT|           

In [ ]:
query = """
    SELECT
        `Loan Category`,
        SUM(`Loan Amount`) AS Total_Loan_Amount,
        AVG(`Income`) AS Average_Income
    FROM loan
    GROUP BY `Loan Category`
"""

spark.sql(query).show()

+------------------+-----------------+------------------+
|     Loan Category|Total_Loan_Amount|    Average_Income|
+------------------+-----------------+------------------+
|           HOUSING|             NULL| 74728.19354838709|
|        TRAVELLING|             NULL| 57016.58490566038|
|       BOOK STORES|             NULL|50903.142857142855|
|       AGRICULTURE|             NULL|60372.666666666664|
|         GOLD LOAN|             NULL| 70838.31506849315|
|  EDUCATIONAL LOAN|             NULL| 62057.64705882353|
|        AUTOMOBILE|             NULL| 68285.63157894737|
|          BUSINESS|             NULL| 70246.54166666667|
|COMPUTER SOFTWARES|             NULL|134376.66666666666|
|           DINNING|             NULL| 67617.54545454546|
|          SHOPPING|             NULL|       50466.34375|
|       RESTAURANTS|             NULL|55228.794871794875|
|       ELECTRONICS|             NULL| 54728.42857142857|
|          BUILDING|             NULL| 69700.16666666667|
|        RESTA

Group by Marital Status, Calculating the Average Income and Maximum Loan Amount

In [ ]:
grouped_loan = loan_data.groupBy('Marital Status').agg(
    {'Income': 'avg', 'Loan Amount': 'max'}
)
grouped_loan.show()

+--------------+-----------------+----------------+
|Marital Status|      avg(Income)|max(Loan Amount)|
+--------------+-----------------+----------------+
|       MARRIED|71465.57846153846|         999,698|
|        SINGLE|61234.74825174825|         964,109|
+--------------+-----------------+----------------+



In [ ]:
query = """
SELECT `Marital Status`,
       AVG(Income) AS Avg_Income,
       MAX(`Loan Amount`) AS Max_Loan_Amount
FROM loan
GROUP BY `Marital Status`
"""
spark.sql(query).show()

+--------------+-----------------+---------------+
|Marital Status|       Avg_Income|Max_Loan_Amount|
+--------------+-----------------+---------------+
|       MARRIED|71465.57846153846|        999,698|
|        SINGLE|61234.74825174825|        964,109|
+--------------+-----------------+---------------+

